# Algotytmy obliczania otoczki wypukłej krok po kroku

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bitalg.visualizer.main import Visualizer
from functools import cmp_to_key

## Wczytywanie, zapisywanie zbiorów punktów do plikow

In [9]:
def save_points(f_path, points):
    with open(f_path, "w") as f:
        f.write(f'{points}')

def load_points(f_path):
    with open(f_path, "r") as f:
        return eval(f.readlines()[0])

## Funkcje wyznacznika, orientacji oraz odległości

In [92]:
def det(a, b, c):
    return ((b[1]-a[1]) * (c[0] - b[0]) - (b[0] - a[0]) * (c[1] - b[1]))

def points_orientation(a, b, c, eps = 0):
    computed_det = det(a, b, c)
    if computed_det > eps: return 1
    elif computed_det < -eps: return -1
    return 0

def points_distance_sqare(a, b):
    return ((a[0]-b[0])**2 + (a[1]-b[1])**2)

## Algorytmy znajdowania otoczki wypukłej

Wszyskie zaimplementowane poniżej algorytmy przyjmują listę punktów w postaci [(x1, y1)...] oraz zwracają listę kolejnych punktów otoczki postaci [(x1, y1)...]


### Algorytm Grahama

Algorytm Grahama tworzy otoczkę wypukłą poprzez utrzymywanie stosu, w którym znajdują się punkty, które mogą, ale nie muszą tworzyć otoczki wypukłej. Za każdym razem jest wstawiany na stos jeden punkt z zbioru punktów i jest on usuwany ze stosu, jeżeli nie jest punktem otoczki. Kiedy algorytm kończy się, stos zawiera tylko punkty otoczki wypukłej w kierunku przeciwnym do ruchu wskazówek zegera.

Złożoność czasowa O(nlogn)

In [4]:
def graham_algorithm(points):

    lowest_y_point = points[0]
    for i in range(len(points)): 
        if points[i][1] < lowest_y_point[1] or (points[i][1] == lowest_y_point[1] and points[i][0] < lowest_y_point[0]): lowest_y_point = points[i] 

    def side_comp(x, y):
        orientation = points_orientation(lowest_y_point, x, y)
        if orientation == 0: return points_distance_sqare(lowest_y_point, x) - points_distance_sqare(lowest_y_point, y)
        return orientation

    points.sort(key=cmp_to_key(side_comp))

    new_points = [lowest_y_point]

    for i in range(1, len(points)):
        if i == len(points)-1 or points_orientation(lowest_y_point, points[i], points[i+1]) != 0: new_points.append(points[i])

    S = [new_points[0]]
    
    for i in range(1, len(new_points)):
        while len(S) > 1 and points_orientation(S[-2], S[-1], new_points[i]) >= 0: S.pop()

        S.append(new_points[i])

    return S

### Algorytm Jarvisa

Algorytm Jarvisa oblicza otoczkę wypukłą dla zbioru punktów przez technikę zwaną owijaniem paczki (gift wraping). Algorytm Jarvisa buduje sekwencję, w której każdy kolejny punkt ma najmniejszy kąt w odniesieniu do poprzedniego. Jeśli punkty mają ten sam kąt, wybieramy dalszy.

Złożoność czasowa O(nh) (gdzie h to wielkość zbioru wynikowego)

In [5]:
def jarvis_algorithm(points):
    on_chain = points[0]
    for i in range(len(points)): 
        if points[i][1] < on_chain[1] or (points[i][1] == on_chain[1] and points[i][0] < on_chain[0]): on_chain = points[i] 

    S = []
    while not S or on_chain != S[0]:
        S.append(on_chain)
        next = points[0]
        for point in points: 
            if next == on_chain or points_orientation(on_chain, next, point) == 1 or (points_orientation(on_chain, next, point) == 0 and points_distance_sqare(on_chain, next)< points_distance_sqare(on_chain, point)):
                next = point
        on_chain = next

    return S

### Algorytm Dziel i Zwyciężaj

Algorytm ten polega na rekurencyjnym podziale zbioru na dwie części, po których przeprocesowaniu następuje ich łączenie. Podział wykonujemy wedle pionowej prostej przechodzącej przez element środkowy względem wspł. x. W tym celu na początu sortujemy cały zbiór punktów. W momencie gdy zbiór zawiera jedynie <= 5 punktów, Wykonujemy brutalny algorytm wyszukiwania otoczki.

Łączenie odbywa się poprzez wybranie dla lewej otoczki punktu najbardziej wysuniętego w prawo p i dla prawej otoczki punktu najbardziej wysuniętego w lewo q. Następnie tworzymy dwa odcinki pq. Jeden z nich będziemy przesuwać w górę, a drugi w dół aż do utworzenia dolnej i górnej stycznej.

Złożoność O(nlogn)

In [6]:
# Driver code
def divide_and_conquer(points):
    return divide_and_conquer_main_func(sorted(points))

# Main function
def divide_and_conquer_main_func(points):
    if len(points) <= 5: return jarvis_algorithm(points)

    left_part = divide_and_conquer_main_func(points[:len(points)//2 + 2])
    right_part = divide_and_conquer_main_func(points[len(points)//2 + 2:])

    return merge(left_part, right_part)

# Merge function
def merge(a, b):

    if len(a) == 0: return b
    elif len(b) == 0: return a

    max_right_ind = 0
    for i in range(len(a)): 
        if a[i][0] > a[max_right_ind][0]: max_right_ind = i 

    max_left_ind = 0
    for i in range(len(b)): 
        if b[i][0] < b[max_left_ind][0]: max_left_ind = i 

    top_a_ind = bottom_a_ind = max_right_ind
    top_b_ind = bottom_b_ind = max_left_ind

 
    while True:
        done = True
        while points_orientation(b[top_b_ind], a[top_a_ind], a[(top_a_ind + 1)%len(a)]) != -1: 
            top_a_ind = (top_a_ind + 1)%len(a)

        if points_orientation(a[top_a_ind], b[top_b_ind], b[(top_b_ind - 1 + len(b))%len(b)]) != 1: 
            top_b_ind = (top_b_ind - 1 + len(b))%len(b)
            done = False
            
        if done: break
        
    while True:
        done = True
        while points_orientation(b[bottom_b_ind], a[bottom_a_ind], a[(bottom_a_ind - 1 + len(a))%len(a)]) != 1: 
            bottom_a_ind = (bottom_a_ind - 1 + len(a))%len(a)

        if points_orientation(a[bottom_a_ind], b[bottom_b_ind], b[(bottom_b_ind + 1)%len(b)]) != -1: 
            bottom_b_ind = (bottom_b_ind + 1)%len(b)
            done = False

        if done: break

    new_points = []
    i = top_a_ind
    while i != bottom_a_ind: 
        new_points.append(a[i])
        i = (i + 1) % len(a)
    new_points.append(a[bottom_a_ind])

    i = bottom_b_ind
    while i != top_b_ind: 
        new_points.append(b[i])
        i = (i + 1) % len(b)
    new_points.append(b[top_b_ind])

    return new_points

### Algorytm Chana

Algorytm Chana jest optymalnym względem zbioru wynikowego algorytmem znajdowania otoczki wypukłej. Oznacza to tyle, że niejako łącząc algorytm Grahama i Jarvisa łączy podejście O(nlogn) zależne od ilości danych z podejściem O(nh) zależnym od mnogości zbioru wynikowego. Polega on na stworzeniu pomiejszych otoczek podzbiorów punktów, a następnie na ich połączeniu.

Złożoność obliczeniowa O(nlogh)

In [104]:
def chan_algorithm(points):
    def most_acute(hull, a):
        left, right = -1, len(hull)

        while left + 1 < right:
            mid = (left + right) // 2
            if points_orientation(a, hull[(mid + len(hull) - 1) % len(hull)], hull[mid]) != -1:
                left = mid
            else:
                right = mid

        return hull[left]


    min_y_ind = 0
    for i in range(len(points)):
        if points[i][1] < points[min_y_ind][1]: min_y_ind = i


    for t in range(len(points)):
        m = 2 ** (2 ** t)
        hulls = [graham_algorithm(points[i:i+m]) for i in range((len(points)+m-1)//m)]

        if len(hulls) == 1: return hulls[0]

        ans_hull = [points[min_y_ind]]

        # pseudo jarvis
        for i in range(m):
            on_chain = points[min_y_ind]
            next = most_acute(hulls[0], on_chain)
            for j in range(1, len(hulls)):
                candidate = most_acute(hulls[j], on_chain)
                if candidate == on_chain or points_orientation(on_chain, next, candidate) == 1 or (points_orientation(on_chain, next, candidate) == 0 and points_distance_sqare(on_chain, next) < points_distance_sqare(on_chain, candidate)):
                    next = candidate
            on_chain = next
        if on_chain == ans_hull[0]: return ans_hull       

    return []        

In [111]:
points = [
        (-62.781083483620016, 9.295526540248986),
        (-10.543100198806997, -26.080520917553812),
        (-81.64932184252287, -74.42163273030921),
        (-36.297317058417946, -72.91194239793609),
        (37.795092197502356, 57.71110085986143),
        (62.511149567563905, -29.172821102708937),
        (21.82806671019955, 2.647377124715007),
        (-46.24539555503924, 42.65521594922478),
        (-77.92302295134137, -7.666110427206263),
        (25.85862324263843, 62.49564419388622),
        (-27.71649622636616, -67.33453457840331),
        (94.83039177581244, -55.52473300629532),
        (-26.29675918891381, -51.30150933048958),
        (-70.11654929355294, 16.723865705806816),
        (26.682887992598097, -65.55763984116587),
        (57.03377667841906, -55.56635171240132),
        (-16.053624841650247, -42.333295668531456),
        (-56.810858686395505, -37.41219002465095),
        (15.604076302407279, -24.85779870929437),
        (-71.77261869976445, -12.306083264402673)]

print(len(graham_algorithm(points)), (jarvis_algorithm(points)), (chan_algorithm(points)))

9 [(-81.64932184252287, -74.42163273030921), (-36.297317058417946, -72.91194239793609), (26.682887992598097, -65.55763984116587), (94.83039177581244, -55.52473300629532), (37.795092197502356, 57.71110085986143), (25.85862324263843, 62.49564419388622), (-46.24539555503924, 42.65521594922478), (-70.11654929355294, 16.723865705806816), (-77.92302295134137, -7.666110427206263)] [(-81.64932184252287, -74.42163273030921), (-36.297317058417946, -72.91194239793609), (26.682887992598097, -65.55763984116587), (94.83039177581244, -55.52473300629532), (37.795092197502356, 57.71110085986143), (25.85862324263843, 62.49564419388622), (-46.24539555503924, 42.65521594922478), (-70.11654929355294, 16.723865705806816), (-77.92302295134137, -7.666110427206263)]
